In [3]:
import numpy as np
import pandas as pd
from time import perf_counter
from sklearn.datasets import load_iris, fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LinearDA
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

def get_datasets():
    iris_data = load_iris()
    X_iris, y_iris = iris_data.data, iris_data.target
    mnist_data = fetch_openml('mnist_784', version=1, return_X_y=True, as_frame=False, parser='auto')
    X_mnist_scaled, y_mnist = mnist_data[0] / 255.0, mnist_data[1]
    X_mnist_small, _, y_mnist_small, _ = train_test_split(
        X_mnist_scaled, y_mnist, train_size=5000, stratify=y_mnist, random_state=42
    )
    return {'Iris': (X_iris, y_iris), 'MNIST': (X_mnist_small, y_mnist_small)}

def reduce_dimensions(X, y, method, name):
    n_lda_components = min(len(np.unique(y)) - 1, X.shape[1])
    reducers = {
        'PCA': PCA(n_components=min(50, X.shape[1])),
        'LDA': LinearDA(n_components=n_lda_components),
        'KernelPCA': KernelPCA(n_components=min(50, X.shape[1]), kernel='rbf', gamma=0.05)
    }
    reducer = reducers.get(method)
    if not reducer:
        raise ValueError(f"Dimensionality reduction method '{method}' is not supported.")
    return reducer.fit_transform(X, y) if method == 'LDA' else reducer.fit_transform(X)

def assess_classifier(X, y, clf):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    start_time = perf_counter()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    end_time = perf_counter()
    return {
        'Accuracy': accuracy_score(y_test, y_pred),
        'F1 Score': f1_score(y_test, y_pred, average='weighted'),
        'Training Time': end_time - start_time
    }

def conduct_experiments():
    datasets = get_datasets()
    classifiers = {'Iris': RandomForestClassifier(random_state=42), 'MNIST': SVC(kernel='rbf')}
    experiment_results = []

    for name, (X, y) in datasets.items():
        for method in ['PCA', 'LDA', 'KernelPCA']:
            X_reduced = reduce_dimensions(X, y, method, name)
            clf = classifiers[name]
            performance = assess_classifier(X_reduced, y, clf)
            performance.update({'Dataset': name, 'Method': method})
            experiment_results.append(performance)

    return pd.DataFrame(experiment_results)

if __name__ == "__main__":
    experiment_data = conduct_experiments()
    print(experiment_data.pivot(index='Method', columns='Dataset', values=['Accuracy', 'F1 Score', 'Training Time']).to_string())


          Accuracy           F1 Score           Training Time          
Dataset       Iris     MNIST     Iris     MNIST          Iris     MNIST
Method                                                                 
KernelPCA      1.0  0.921333      1.0  0.921079      0.204380  0.635451
LDA            1.0  0.931333      1.0  0.931202      0.151579  0.247583
PCA            1.0  0.952667      1.0  0.952614      0.150352  0.631832
